In [2]:
import cv2
import numpy as np
from PIL import Image
import pytesseract
import re

# Set up the Tesseract executable path (adjust the path if necessary)
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\Admin\AppData\Local\Programs\TesseractOCR\tesseract.exe'




# Load the image
image_path = 'sample1.jpeg'
image = cv2.imread(image_path)

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply sharpening filter
kernel = np.array([[0, -1, 0], 
                   [-1, 5,-1], 
                   [0, -1, 0]])
sharpened = cv2.filter2D(gray, -1, kernel)

# Apply edge detection
edges = cv2.Canny(sharpened, 50, 150, apertureSize=3)


# # Apply edge detection
# edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Sort contours by area (largest to smallest) and take the largest one
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:1]

for contour in contours:
    # Get the bounding box of the largest contour
    x, y, w, h = cv2.boundingRect(contour)
    # Crop the image to the bounding box
    cropped_image = image[y:y+h, x:x+w]

# Convert the cropped image to PIL format for OCR
cropped_image_pil = Image.fromarray(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))

# Perform OCR
text = pytesseract.image_to_string(cropped_image_pil)

print("Extracted Text:")
print(text)


Extracted Text:
Certificate of Virtual Internship

This isto certify that

VISHAKHA SANJEEV SHINDE

Intemational Institute of Information Technology (1A?'T)

has successfully completed 10 week
AML Virtual Internship
During January - March 2024

Supported By : India Edu Program

Google for Developer




In [6]:
# Define dictionaries to store labels and regex patterns
entity_labels = {
  "name": r'\b[A-Z][a-zA-Z\'-]\s[A-Z][a-zA-Z\'-]\b(?:\s[A-Z][a-zA-Z\'-]*)?',
  "institute": r"\(([^)]+)\)",  # Capture institute name only
  "internship": r"\d+ weeks of [A-Za-z\s\-]+Internship",
  "dates": r"During\s+(.+?)\s+20\d{2}",
  "organizer": r"(?:By|Supported By):\s+(.+)",
  "issued_by": r"([A-Z][a-z]+\s+\(.*\))",  # Capture everything within parentheses
  "approved_by": r"Dr\.\s+([A-Z][a-z]+)\s+\(.*\)"  # Capture everything within parentheses
}

# Extract entities
extracted_entities = {}
for label, pattern in entity_labels.items():
  match = re.search(pattern, text, re.MULTILINE)  # Use re.MULTILINE for multi-line text
  if match:
    try:
      extracted_entities[label] = match.group(1)  # Capture group 1 (extracted text)
    except IndexError:  # Handle cases where the group doesn't exist
      # Optional: Provide informative message or default value
      print(f"Warning: No group 1 captured for label '{label}'.")
      extracted_entities[label] = None  # Or set a default value

# Print extracted entities with labels
for label, entity in extracted_entities.items():
  print(f"{label}: {entity}")


institute: IA#IT
dates: January - March
issued_by: Technology (IA#IT)


In [7]:

# Example text
# text = "Acme Corporation Ltd, Globex Inc. Ltd, Stark Industries Pvt Ltd, Stark Industries Offered by Stark Industries"

# Regex pattern to match company name with various keywords
regex = r'(\w+(?:\s\w+)*)\s(?:Inc\.\s)?(?:Pvt\s)?(?:Offered\sby\s)?Ltd'

# Find all matches in the text
matches = re.findall(regex, text, re.IGNORECASE)

# Output the company names
for company in matches:
    print(company)

In [8]:
pattern = r'Dr\.\s+([A-Z][a-z]+(?:\s[A-Z][a-z]+)*)'
pattern1 = r'(\b\w+(?:\s\w+)*)\s(?:Inc\.\s)?(?:Pvt\s(?:Ltd)?\b)?(?:Offered\sby\s)?(?:Ltd|Corp\b)'

# Find all matches
matches = re.findall(pattern, text)

# Print the matches
for match in matches:
    print(f"Found: {match}")

In [9]:
# Regex pattern to match date range
date_pattern = r'period of\s+(\d{1,2} [A-Za-z]+ \d{4})\s*-\s*(\d{1,2} [A-Za-z]+ \d{4})'

# Find all matches in the text
matches = re.findall(date_pattern, text)

# Output the results
for match in matches:
    start_date, end_date = match
    print(f"Date range: From {start_date} to {end_date}")

In [10]:
# Regex pattern to match durations
regex = r'\b(?:(\d+)\s*weeks|(\d+)\s*months|tenure\s+of\s+(\d+)|of\s+(\d+))\b'

# Find all matches in the text
matches = re.findall(regex, text)

# Output the matches
for match in matches:
    if match[0]:
        print(f"{match[0]} weeks")
    elif match[1]:
        print(f"{match[1]} months")
    elif match[2]:
        print(f"tenure of {match[2]} months")
    else:
        print(f"of {match[3]} months")

10 weeks


In [11]:
 def extract_name(text):
    """Function to extract name from resume text using spaCy's Matcher."""
    nlp = spacy.load('en_core_web_sm')  # Load English language model
    matcher = Matcher(nlp.vocab)  # Initialize Matcher with spaCy's vocabulary

    # Define name patterns for Matcher
    patterns = [
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}],  # First name and Last name
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}],  # First name, Middle name, and Last name
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}]  # First name, Middle name, Middle name, and Last name
        # Add more patterns as needed
    ]

    for pattern in patterns:
        matcher.add('NAME', patterns=[pattern])  # Add each pattern to the Matcher

    doc = nlp(text)  # Process the resume text with spaCy
    matches = matcher(doc)  # Apply the Matcher to find name patterns

    for match_id, start, end in matches:
        span = doc[start:end]  # Get the matched span of text
        return span.text  # Return the matched text (name)

    return None  # Return None if no name is found


In [13]:
def extract_proper_nouns(text):
    """Function to extract proper nouns from text using SpaCy."""
    nlp = spacy.load("en_core_web_sm")  # Load SpaCy model
    doc = nlp(text)  # Process the text with SpaCy

    # Extract proper nouns
    proper_nouns = [token.text for token in doc if token.pos_ == "PROPN"]
    return proper_nouns

In [14]:
# Extract name
name = extract_name(text)
print("Extracted Name:", name)

# Extract proper nouns
proper_nouns = extract_proper_nouns(text)
print("Proper Nouns:", proper_nouns)

NameError: name 'spacy' is not defined